# In-depth Analysis of Successful Negotiation Cases

View complete details of successful negotiations, including:
- Complete conversation transcript
- Deal Terms reached
- Memory (M) state
- Plan (P) strategy
- Negotiation process analysis

## 1. Setup and Imports

In [29]:
import sqlite3
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

print("✅ Import successful")

✅ Import successful


## 2. Connect to Database

In [30]:
# Modify the path here
DB_PATH = "negotiations (1).db"  # Change to your database path

conn = sqlite3.connect(DB_PATH)
print(f"✅ Connected to database: {DB_PATH}")

✅ Connected to database: negotiations (1).db


## 3. List All Successful Negotiations

In [31]:
# Get all successful negotiations
query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    student_role,
    ai_model,
    use_memory,
    use_plan,
    created_at,
    feedback_text TEXT
FROM negotiation_sessions
WHERE deal_reached = 1
ORDER BY created_at DESC
"""

successful_df = pd.read_sql_query(query, conn)

print(f"\n📊 Found {len(successful_df)} successful negotiations\n")

# Display list
display_df = successful_df.copy()
display_df['session_id_short'] = display_df['session_id'].str[:8]
display_df['M+P'] = display_df.apply(
    lambda x: '✅' if x['use_memory'] and x['use_plan'] else '❌', 
    axis=1
)

display(display_df[['session_id_short', 'student_name', 'scenario_name', 
                     'ai_model', 'M+P', 'created_at']])

print("\n💡 Copy session_id_short to view details")


📊 Found 13 successful negotiations



,session_id_short,student_name,scenario_name,ai_model,M+P,created_at
0,21ea396e,Chris,Main_Street,openai/gpt-5,❌,2025-12-02T11:32:50.719142
1,351522eb,Chris,Main_Street,openai/gpt-5,✅,2025-12-02T11:14:31.145454
2,6bcbd0fe,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:59:07.400156
3,03b5791c,Chris,Main_Street,openai/gpt-5,✅,2025-12-01T06:54:53.616963
4,06eb1797,Chris,Top_talent,openai/gpt-5,❌,2025-12-01T06:54:01.765574
5,d9e5ab85,Chris,Top_talent,openai/gpt-5,❌,2025-12-01T06:53:49.801746
6,2d6e0616,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:17:38.355369
7,30c3c80c,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:17:23.112858
8,746dbd3e,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:57.375665
9,05a9b9d7,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:51.933084



💡 Copy session_id_short to view details


## 4. View Specific Case Details

### 👇 Enter the session_id you want to view below (first 8 characters are sufficient)

In [32]:
# ============================================
# 👉 MODIFY HERE: Enter the session_id you want to view
# ============================================
SESSION_ID = "9c8469c8-9edc-4a23-af46-3a07c46fc90f"  # Change to the actual session_id (first 8 characters)

# If only first 8 characters, find the complete ID
if len(SESSION_ID) == 8:
    full_id_query = f"""
    SELECT session_id FROM negotiation_sessions 
    WHERE session_id LIKE '{SESSION_ID}%' AND deal_reached = 1
    """
    result = pd.read_sql_query(full_id_query, conn)
    if len(result) > 0:
        SESSION_ID = result['session_id'].iloc[0]
        print(f"✅ Found complete ID: {SESSION_ID}")
    else:
        print(f"❌ No successful cases found starting with {SESSION_ID} ")

# Get complete data
query = f"""
SELECT * FROM negotiation_sessions
WHERE session_id = '{SESSION_ID}'
"""

case_df = pd.read_sql_query(query, conn)

if len(case_df) == 0:
    print(f"❌ session_id not found: {SESSION_ID}")
else:
    case = case_df.iloc[0]
    print(f"\n✅ Case loaded: {SESSION_ID}")


✅ Case loaded: 9c8469c8-9edc-4a23-af46-3a07c46fc90f


## 5. 📋 Case Overview

In [22]:
if len(case_df) > 0:
    print("=" * 80)
    print("                          Case Overview")
    print("=" * 80)
    
    print(f"\n🆔 Session ID:     {case['session_id']}")
    print(f"👤 Student:           {case['student_name']} (ID: {case['student_id']})")
    print(f"🎯 Scenario:           {case['scenario_name']}")
    print(f"🎭 Student Role:       {case['student_role']}")
    print(f"🤖 AI Role:         {case['ai_role']}")
    print(f"🧠 AI Model:         {case['ai_model']}")
    print(f"🔧 Memory Enabled:     {'Yes ✅' if case['use_memory'] else 'No ❌'}")
    print(f"📋 Plan Enabled:       {'Yes ✅' if case['use_plan'] else 'No ❌'}")
    print(f"🎲 Who Starts First:       {'student' if case['student_goes_first'] else 'AI'}")
    print(f"🔄 Total Rounds:       {case['total_rounds']}")
    
    # 计算时长
    try:
        start = pd.to_datetime(case['created_at'])
        end = pd.to_datetime(case['updated_at'])
        duration = (end - start).total_seconds() / 60
        print(f"⏳ Duration:           {duration:.1f} minutes")
    except:
        pass
    
    # 消息统计
    try:
        transcript = json.loads(case['transcript'])
        print(f"💬 Total Messages:       {len(transcript)}")
        print(f"📅 Start Time:       {case['created_at']}")
        print(f"✅ Completion Time:       {case['updated_at']}")
    except:
        pass

                          Case Overview

🆔 Session ID:     03b5791c-d8c7-480f-9810-db0f6653bd45
👤 Student:           Chris (ID: 123)
🎯 Scenario:           Main_Street
🎭 Student Role:       side2
🤖 AI Role:         side1
🧠 AI Model:         openai/gpt-5
🔧 Memory Enabled:     Yes ✅
📋 Plan Enabled:       Yes ✅
🎲 Who Starts First:       student
🔄 Total Rounds:       6
⏳ Duration:           36.3 minutes
💬 Total Messages:       13
📅 Start Time:       2025-12-01T06:54:53.616963
✅ Completion Time:       2025-12-01T07:31:09.857781


## 6. 🤝 Deal Terms Reached

In [33]:
if len(case_df) > 0:
    print("=" * 80)
    print("                          Deal Terms")
    print("=" * 80)
    
    print("\n📝 Student's Proposed Deal Terms:")
    print("-" * 80)
    if case['student_deal_json']:
        try:
            student_deal = json.loads(case['student_deal_json'])
            print(json.dumps(student_deal, indent=2, ensure_ascii=False))
        except Exception as e:
            print(f"Parse error: {e}")
    else:
        print("(Not recorded)")
    
    print("\n\n🤖 AI Confirmed Deal Terms:")
    print("-" * 80)
    if case['ai_deal_json']:
        try:
            ai_deal = json.loads(case['ai_deal_json'])
            print(json.dumps(ai_deal, indent=2, ensure_ascii=False))
        except Exception as e:
            print(f"Parse error: {e}")
    else:
        print("(Not recorded)")
    
    # 对比差异
    if case['student_deal_json'] and case['ai_deal_json']:
        try:
            student_deal = json.loads(case['student_deal_json'])
            ai_deal = json.loads(case['ai_deal_json'])
            
            print("\n\n🔍 Comparison Analysis:")
            print("-" * 80)
            
            all_keys = set(student_deal.keys()) | set(ai_deal.keys())
            
            for key in sorted(all_keys):
                student_val = student_deal.get(key, "(Missing)")
                ai_val = ai_deal.get(key, "(Missing)")
                
                if student_val == ai_val:
                    print(f"✅ {key:30s}: {student_val}")
                else:
                    print(f"⚠️  {key:30s}:")
                    print(f"   Student: {student_val}")
                    print(f"   AI:   {ai_val}")
        except:
            pass

                          Deal Terms

📝 Student's Proposed Deal Terms:
--------------------------------------------------------------------------------
(Not recorded)


🤖 AI Confirmed Deal Terms:
--------------------------------------------------------------------------------
(Not recorded)


In [34]:
case["ai_memory_history"]

'[{"round": "2.1", "content": "OFFERS: \\nUs: Start date: July 5 [unchanged]; Work location: New Haven, CT [unchanged]; Base salary: $135,000 [unchanged]; Remote work: 0 days/week [unchanged]; Signing bonus: $0 [unchanged]; Vacation: 2 weeks [unchanged]; Blair rotation: None [unchanged]. Alt exists (Option B): Start date July 15; Remote 1 day/week; Signing bonus $5,000; others same.\\nThem: Start date: August 1 [first offer]; Work location: Philadelphia, PA [first offer]; Base salary: $165,000 [first offer]; Remote work: 0 days/week [first offer]; Signing bonus: $40,000 [first offer]; Vacation: 4 weeks [first offer]; Blair rotation: Formal 3 months in DC, then Philadelphia [first offer].\\nThem-best-for-us: Start date August 1; Location Philadelphia; Base $165,000; Remote 0; Signing bonus $40,000; Vacation 4 weeks; Blair rotation 3 months DC then Philadelphia.\\n\\nOPPONENT PATTERNS: \\n- Firm anchors on Aug 1+ start and 3-month DC rotation; tied rotation to enthusiasm. Accepted our 0 

In [35]:
case["ai_plan_history"]

'[{"round": "2.1", "content": "- ROUND GOAL: Win New Haven as post-rotation home base, keep 0 remote, land $140k base with \\u2264$10k sign-on, and set 2.5 weeks vacation, while accepting Aug 1 start + 3-month DC rotation.\\n- KEY LEVERS: Location (push New Haven; Seoul as fallback if Philly immovable), Salary (135\\u2013140k), Signing bonus (0\\u201315k; move this before salary), Vacation (2\\u20132.5 weeks; hold 3 weeks as later stretch), Remote (hold 0), Start date (accept Aug 1), Blair rotation (accept 3 months; no extension).\\n- TACTICS: Open by acknowledging Aug 1 + DC rotation; explicitly tie that acceptance to New Haven home base post-rotation; anchor comp at $140k + $10k, 0 remote, 2.5 weeks vacation; use contingent framing (\\u201cIf New Haven + 2.5 weeks, we commit to Aug 1 + 3-month DC rotation\\u201d); probe willingness for New Haven for at least 18 months after rotation; signal bonus is the flexible knob before touching salary/vacation; justify with team needs/equity; av

## 7. 🧠 Memory (M) State

In [36]:
if len(case_df) > 0:
    print("=" * 80)
    print("                     AI Memory (Final State)")
    print("=" * 80)
    
    if not case['use_memory']:
        print("\n❌ Memory not enabled for this case")
    elif not case['ai_memory']:
        print("\n(Memory state not recorded)")
    else:
        print("\n🧠 AI Memory recorded the following information:\n")
        print(case['ai_memory'])
        
        # 分析Memory内容
        memory_text = case['ai_memory']
        print("\n\n📊 Memory Analysis:")
        print("-" * 80)
        print(f"Character count: {len(memory_text)}")
        print(f"Line count: {len(memory_text.split('\n'))}")
        
        # 检查关键词
        keywords = ['OFFERS', 'PATTERNS', 'PRIORITIES', 'CONSTRAINTS']
        found = [kw for kw in keywords if kw in memory_text]
        print(f"Key sections included: {', '.join(found)}")

                     AI Memory (Final State)

🧠 AI Memory recorded the following information:

OFFERS: [Us: Start: Aug 1 [unchanged]; Location: Philadelphia [unchanged]; Base: $135k→$150k [updated]; Remote: 0/wk [unchanged]; Bonus: $30k [unchanged] (alt: stretch to $32.5k to close today); Vacation: 2.5→3.0 wks [updated]; Blair: 3-mo DC [unchanged].]
[Them: Start: Aug 1 [unchanged]; Location: Philadelphia [unchanged]; Base: $150k [unchanged]; Remote: 0/wk [unchanged]; Bonus: $40k [unchanged]; Vacation: 3.5→3.0 wks [updated] (will do 3.0 wks for $40k sign-on); Blair: 3-mo DC [unchanged].]
[Them-best-for-us: Aug 1; Philadelphia; Base $150k; Remote 0/wk; Bonus $40k; Vacation 3.0 wks; Blair 3-mo DC.]

OPPONENT PATTERNS: [Accepted Aug 1, 3-mo DC rotation, Philadelphia, 0 remote; Conceded base 165k→150k; Conceded vacation 4.0→3.5→3.0 wks (3.0 only if $40k bonus); Holding bonus at $40k; Will sign immediately if sign-on is $40k with our $150k base and 3.0 wks.]

OPPONENT PRIORITIES: [Philadelph

## 8. 📋 Plan (P) Strategy

In [37]:
if len(case_df) > 0:
    print("=" * 80)
    print("                   AI Strategic Plan (Final State)")
    print("=" * 80)
    
    if not case['use_plan']:
        print("\n❌ Plan not enabled for this case")
    elif not case['ai_plan']:
        print("\n(Plan state not recorded)")
    else:
        print("\n📋 AI's Final Strategic Plan:\n")
        print(case['ai_plan'])
        
        # 分析Plan内容
        plan_text = case['ai_plan']
        print("\n\n📊 Plan Analysis:")
        print("-" * 80)
        print(f"Character count: {len(plan_text)}")
        print(f"Line count: {len(plan_text.split('\n'))}")
        
        # 检查关键词
        keywords = ['ROUND GOAL', 'KEY LEVERS', 'TACTICS', 'OFFER SCAFFOLD', 'RISK']
        found = [kw for kw in keywords if kw in plan_text]
        print(f"Strategy sections included: {', '.join(found)}")

                   AI Strategic Plan (Final State)

📋 AI's Final Strategic Plan:

- ROUND GOAL: Close now on Aug 1 / Philadelphia / 3‑mo DC / 0 remote / Base $150k with Vacation 3.0 wks and Signing Bonus $35k (target) or $37.5k (floor); do not agree to $40k + 3.0 wks.
- KEY LEVERS: Signing Bonus (primary); Vacation (secondary—only trade down to 2.5 wks if Bonus = $40k). Others fixed per their constraints (start, location, rotation, remote, base).
- TACTICS: Open with full alignment on their must‑haves; cite internal equity/band limits; present a clear “choose‑and‑sign now” menu; set today-only approval; refuse $40k + 3.0 wks as beyond approval.
- TACTICS: If pushback, reaffirm $37.5k is the absolute ceiling at 3.0 wks; offer $40k only paired with 2.5 wks; otherwise pause (no deal > sub‑BATNA deal).
- OFFER SCAFFOLD: Option A (preferred): Start Aug 1; Location Philadelphia; Base $150,000; Remote 0/wk; Signing Bonus $35,000; Vacation 3.0 wks; Blair Rotation 3 months.
- OFFER SCAFFOLD: Op

## 9. 💬 Complete Conversation Transcript（Transcript）

In [38]:
if len(case_df) > 0:
    print("=" * 80)
    print("                         Complete Conversation Transcript")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        total_messages = len(transcript)
        
        print(f"\n💬 共 {total_messages} 条消息\n")
        print("=" * 80)
        
        for i, message in enumerate(transcript, 1):
            # 解析消息格式: "Round X.Y - Label: content"
            if " - " in message:
                parts = message.split(" - ", 1)
                round_info = parts[0]
                
                if len(parts) > 1 and ":" in parts[1]:
                    speaker_content = parts[1].split(":", 1)
                    speaker = speaker_content[0].strip()
                    content = speaker_content[1].strip() if len(speaker_content) > 1 else ""
                    
                    # 判断是学生还是AI
                    if case['student_role'] in speaker or "Student" in speaker:
                        icon = "👤"
                        print(f"\n{icon} [{round_info}] {speaker}:")
                    else:
                        icon = "🤖"
                        print(f"\n{icon} [{round_info}] {speaker}:")
                    
                    # 打印内容（处理长消息）
                    if len(content) > 500:
                        print(f"   {content[:2000]}...")
                        print(f"   ... (共 {len(content)} characters)")
                    else:
                        print(f"   {content}")
                else:
                    print(f"\n💬 [{round_info}]")
                    print(f"   {parts[1] if len(parts) > 1 else message}")
            else:
                print(f"\n💬 消息 {i}:")
                if len(message) > 500:
                    print(f"   {message[:500]}...")
                    print(f"   ... (共 {len(message)} characters)")
                else:
                    print(f"   {message}")
            
            print("-" * 80)
        
    except Exception as e:
        print(f"\n❌ 解析transcript出错: {e}")

                         Complete Conversation Transcript

💬 共 13 条消息


🤖 [Round 1.1] The Recruiter:
   Thanks for spending the time today. We’re excited about you and want to make this feel like a great landing while setting you up to have outsized impact quickly.

Here’s a strong package proposal that aligns with how we’ll get you in front of the right people and moving fast:

Option A (fastest impact)
- Start date: July 5 (you’ll be part of the CEO’s internal online conference in your first week)
- Work location: New Haven, CT (closest collaboration with product, data, and brand—this is where the need and impact are highest)
- Base salary: $135,000 (aligned with our internal equity banding for Directors)
- Remote policy: On-site (0 days/week) to start—tight mentorship, team leadership, and onboarding velocity
- Signing bonus: $0
- Vacation: 2 weeks annually
- Blair rotation: Not included; we’ll set up a standing cadence with Blair and DC stakeholders without a formal rotation so you

In [106]:
case['feedback_text']

'═══════════════════════════════════════\n📊 PERFORMANCE SUMMARY\n═══════════════════════════════════════\n\n- Final outcome: 31,000 points achieved\n- Overall assessment: You secured a deal above your BATNA of 0 points, which is a positive result under time constraints. However, you left significant value on the table by conceding on high-point issues early and not pushing harder on key financial terms.\n\n═══════════════════════════════════════\n🎯 STRATEGIC ANALYSIS\n═══════════════════════════════════════\n\n- **Start Date (15,000 points achieved out of 15,000 possible)**: You secured the maximum value by holding firm on August 1st or later. Your consistent emphasis on personal timing needs paid off as the recruiter conceded early. This was a strong tactical move, ensuring a high-value win without trade-offs.\n  \n- **Work Location (0 points achieved out of 4,000 possible)**: You conceded to New Haven (0 points) despite Philadelphia offering 4,000 points. While you articulated person

## 10. 📊 Negotiation Process Analysis

In [58]:
if len(case_df) > 0:
    print("=" * 80)
    print("                        Negotiation Process Analysis")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        total_messages = len(transcript)
        
        # 统计消息长度
        message_lengths = []
        student_messages = 0
        ai_messages = 0
        deal_signals = 0
        
        for msg in transcript:
            # 计算消息长度
            if " - " in msg:
                parts = msg.split(" - ", 1)
                if len(parts) > 1:
                    msg_content = parts[1]
                    message_lengths.append(len(msg_content))
            else:
                message_lengths.append(len(msg))
            
            # 统计发言者
            if case['student_role'] in msg:
                student_messages += 1
            elif case['ai_role'] in msg:
                ai_messages += 1
            
            # 检查deal信号
            if "$DEAL_REACHED$" in msg:
                deal_signals += 1
        
        print("\n📊 Basic Statistics:")
        print("-" * 80)
        print(f"Total Messages:        {total_messages}")
        print(f"Student Messages:        {student_messages}")
        print(f"AI Messages:          {ai_messages}")
        print(f"完成Round数:      {total_messages // 2}")
        print(f"$DEAL_REACHED$:  出现 {deal_signals} 次")
        
        if message_lengths:
            print("\n📏 Message Complexity:")
            print("-" * 80)
            print(f"Average Message Length:    {sum(message_lengths)/len(message_lengths):.0f} characters")
            print(f"Shortest Message:        {min(message_lengths)} characters")
            print(f"Longest Message:        {max(message_lengths)} characters")
        
        # 计算每Round平均时长
        try:
            start = pd.to_datetime(case['created_at'])
            end = pd.to_datetime(case['updated_at'])
            total_minutes = (end - start).total_seconds() / 60
            rounds = total_messages // 2
            
            if rounds > 0:
                print("\n⏱️  Time Analysis:")
                print("-" * 80)
                print(f"Total Duration:          {total_minutes:.1f} minutes")
                print(f"每Round平均:      {total_minutes/rounds:.1f} minutes")
                print(f"Average Per Message:    {total_minutes/total_messages:.1f} minutes")
        except:
            pass
        
        # 判断谈判风格
        print("\n🎯 Negotiation Characteristics:")
        print("-" * 80)
        
        avg_length = sum(message_lengths)/len(message_lengths) if message_lengths else 0
        if avg_length > 500:
            print("💬 Message Style:      Detailed (longer messages)")
        elif avg_length > 200:
            print("💬 Message Style:      Moderate")
        else:
            print("💬 Message Style:      Concise (shorter messages)")
        
        if total_minutes / rounds < 3 if rounds > 0 else False:
            print("⚡ Negotiation Pace:      快速（<3minutes/Round）")
        elif total_minutes / rounds < 5 if rounds > 0 else False:
            print("⚡ Negotiation Pace:      Normal（3-5minutes/Round）")
        else:
            print("⚡ Negotiation Pace:      缓慢（>5minutes/Round）")
        
        if rounds < case['total_rounds'] / 2:
            print("🎉 Deal Efficiency:      高（不到一半Round就达成）")
        elif rounds < case['total_rounds']:
            print("🎉 Deal Efficiency:      Normal")
        else:
            print("🎉 Deal Efficiency:      压线（用完所有Round）")
        
    except Exception as e:
        print(f"\n❌ Analysis error: {e}")

## 11. 🔍 Quick View: Comparison of Recent 3 Successful Cases

In [19]:
# 获取最近3个成功案例
recent_query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    ai_model,
    use_memory,
    use_plan,
    transcript,
    created_at,
    updated_at
FROM negotiation_sessions
WHERE deal_reached = 1
ORDER BY created_at DESC
LIMIT 3
"""

recent_df = pd.read_sql_query(recent_query, conn)

if len(recent_df) > 0:
    print("=" * 80)
    print("                 Comparison of Recent 3 Successful Cases")
    print("=" * 80)
    
    comparison_data = []
    
    for idx, row in recent_df.iterrows():
        try:
            transcript = json.loads(row['transcript'])
            msg_count = len(transcript)
            
            start = pd.to_datetime(row['created_at'])
            end = pd.to_datetime(row['updated_at'])
            duration = (end - start).total_seconds() / 60
            
            comparison_data.append({
                'Session': row['session_id'][:8],
                'Student': row['student_name'],
                'Scenario': row['scenario_name'],
                'Model': row['ai_model'].split('/')[-1],
                'M+P': '✅' if row['use_memory'] and row['use_plan'] else '❌',
                'Messages': msg_count,
                'Duration(min)': f"{duration:.1f}",
                'Rounds': msg_count // 2
            })
        except:
            pass
    
    if comparison_data:
        comparison_df = pd.DataFrame(comparison_data)
        display(comparison_df)
else:
    print("\n❌ No successful cases found")

                 Comparison of Recent 3 Successful Cases


,Session,Student,Scenario,Model,M+P,Messages,Duration(min),Rounds
0,6bcbd0fe,Chris,Main_Street,gpt-5,❌,10,26.1,5
1,03b5791c,Chris,Main_Street,gpt-5,✅,13,36.3,6
2,06eb1797,Chris,Top_talent,gpt-5,❌,8,23.0,4


## 12. Close Database Connection

In [13]:
conn.close()
print("✅ Database connection closed")

✅ Database connection closed
